In [ ]:
import pandas as pd
import numpy as np
import akshare as ak
from tqdm import tqdm

In [ ]:
df1 = pd.read_excel("上市公司一览.xlsx", sheet_name="sheet1")
stk_ls = list(df1["证券代码"])
for idx, stk in enumerate(stk_ls):
    stk_ls[idx] = stk.split(".")[0]
print(len(stk_ls))
print(stk_ls)

In [ ]:
stk_name_table = ak.stock_us_spot_em()
print(stk_name_table)

In [ ]:
short_code = [i[4:] for i in stk_name_table.代码.values]
ismatch = [i in stk_ls for i in short_code]
print(len(ismatch))

In [ ]:
stk_in = stk_name_table[ismatch]
print(stk_in.shape)

In [ ]:
stk_in_name = list(stk_in["代码"])
print(stk_in_name)

In [ ]:
df = ak.stock_us_hist(stk_in_name[1], period="daily", start_date="20160101", adjust="qfq")
print(df)

In [ ]:
df["stk_id"] = stk_in_name[1][4:]
print(df)

In [ ]:
df1 = df.rename(columns={
    "stk_id": "stk_id",
    "日期":"trade_date",
    "开盘":"open",
    "收盘":"close",
    "最高":"high",
    "最低":"low",
    "成交量":"volume",
    "成交额":"amount",
    "振幅":"volume",
    "涨跌幅":"pct_change",
    "涨跌额":"change",
    "换手率":"turnover",
})
print(df1.describe())
print(df1)

In [ ]:
data_ls = []
for stk_name in tqdm(stk_in_name):
    df = ak.stock_us_hist(stk_name, period="daily", start_date="20150101", adjust="qfq")
    # df = ak.stock_us_hist(stk_name, period="daily", start_date="20150101", adjust="hfq")
    df["stk_id"] = stk_name[4:]
    df1 = df.rename(columns={
        "stk_id": "stk_id",
        "日期":"trade_date",
        "开盘":"open",
        "收盘":"close",
        "最高":"high",
        "最低":"low",
        "成交量":"volume",
        "成交额":"amount",
        "振幅":"amplitude",
        "涨跌幅":"pct_change",
        "涨跌额":"change",
        "换手率":"turnover",
    })
    # print(np.sum(df1.isna()))
    data_ls.append(df1)

In [ ]:
print(len(data_ls))
print(data_ls[0])

In [ ]:
df2 = pd.concat(data_ls)
print(df2.shape)
df2.to_csv("sp500_raw_data.csv")

In [ ]:
df2 = pd.read_csv("sp500_raw_data.csv")

In [ ]:
df3 = df2.pivot(index='trade_date', columns='stk_id',
                             values=['open', 'close', 'high', 'low', 'volume', 'amount', 'amplitude', 'pct_change', 'change', 'turnover'])
print(df3)

In [ ]:
amount = df3["amount"].replace(0,np.nan)
data1 = {
    "open": df3["open"].replace(0,np.nan),
    "close": df3["close"].replace(0,np.nan),
    "high": df3["high"].replace(0,np.nan),
    "low": df3["low"].replace(0,np.nan),
    "volume": df3["volume"].replace(0,np.nan),
    #
    # "turnover": df3["turnover"].replace(0,np.nan),
}
data1["adv20"] = data1["volume"].rolling(window=20).mean()
data1["vwap"] = amount / data1["volume"]
data1["returns"] = (data1["close"] / data1["close"].shift(1) - 1)
y = (data1["close"] / data1["close"].shift(1) - 1)

In [ ]:
start_date = "2017-07-11"
end_date = "2023-04-13"
# val[(pd.Series(val.index) < start_date) | (pd.Series(val.index) > end_date)].index
for key, val in data1.items():
    data1[key] = val.loc[start_date:end_date,:]
y = y.loc[start_date:end_date,:]

In [ ]:
print(data1["open"].shape)
print(y.shape)

In [ ]:
data2 = data1.copy()
y1 = y.copy()
# index_list = ['open', 'close', 'high', 'low', 'volume', 'adv20', 'vwap', 'returns']
# drop_df = pd.DataFrame(columns=next(iter(data2.values())).columns, index=index_list)
# threshold = next(iter(data2.values())).shape[0] * 0.1
# print(drop_df.shape)
# print(data2["open"].shape)
# print(y.shape)
# for idx, df in data2.items():
#     drop_df.loc[idx] = df.isna().sum() > threshold
# mask = drop_df.sum() > 1
# for idx, df in data2.items():
#     data2[idx] = data2[idx].loc[:, ~mask.values]
#     # data2[idx].fillna(method='ffill',inplace=True)
#     # data2[idx].fillna(method='bfill',inplace=True)
# y = y.loc[:, ~mask.values]
# y.fillna(method='ffill',inplace=True)
# y.fillna(method='bfill',inplace=True)

In [ ]:
# y = (data2["close"] / data2["close"].shift(1) - 1)[1:]
# for idx, df in data2.items():
#     data2[idx] = data2[idx][1:]

In [ ]:
mask = abs(y1) < 0.5
print(data2["open"].shape)
print(y.shape)
for idx, df in data2.items():
    data2[idx] = data2[idx].where(mask, np.nan)
y1 = y1.where(mask, np.nan)
print(data2["open"].shape)
print(y.shape)

In [ ]:
y2 = y1.shift(-1)[:-1]
for idx, df in data2.items():
    data2[idx] = data2[idx][:-1]

In [ ]:
print(data2["returns"])

In [ ]:
print(y2)

In [ ]:
pd.Series(np.nanmax(abs(y2), axis=1)).plot()

In [ ]:
import pickle
with open("us_data_2017_2023.pkl", "wb") as f:
    pickle.dump(data2, f)
with open("us_return_2017_2023.pkl", "wb") as f:
    pickle.dump(y2, f)

In [ ]:
print(y2.shape)
print(data2["open"].shape)